In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [36]:
import pandas as pd

In [37]:
df=pd.read_csv("/content/drive/MyDrive/spotify_millsongdata.csv")

In [38]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [39]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [40]:
df=df.sample(5000).drop('link',axis=1).reset_index(drop=True)

In [41]:
df.head(10)

,artist,song,text
0,Josh Groban,Now Or Never,I watch the morning dawn upon your skin \r\nA...
1,Rihanna,Hypnotized,"Mmm, mmm, ohh, no, oh, no \r\nSomething 'bout..."
2,Fall Out Boy,Alphadog,"Clap until your hands hurt, \r\nStanding ovat..."
3,Used,Take It Away,I'm lying to myself \r\nAnd this dagger is my...
4,Dream Theater,Another Won,It was the mortal passion \r\nThe flame that ...
5,Dean Martin,Just For Fun,Love me love me say you love me \r\nJust for ...
6,Overkill,What's Your Problem,When is it gonna end?! (2x) \r\n \r\nI'm not...
7,New Order,Transmission,"Radio, live transmission \r\nRadio, live tran..."
8,Genesis,For Absent Friends,Sunday at six when they close both the gates ...
9,Amy Grant,How Mercy Looks From Here,"The water rose today, \r\nThe river with the ..."


In [42]:
df=df.sample(3000)

In [43]:
df.shape

(3000, 3)

In [44]:
#Text cleaning or Text preporessing
#cover all text in to lowercase and all \n to replace with empty space
df['text']=df['text'].str.lower().replace(r'^\w\s',' ').replace(r'\n',' ',regex=True)
df

,artist,song,text
3957,Otis Redding,Bring It On Home To Me,if you ever change your mind \r about leaving...
454,Nightwish,Gethsemane,toll no bell for me father \r but let this cu...
431,Cher,Do What You Gotta Do (With Greg Allman),do what you gotta do \r with greg allman \r ...
2601,Train,All American Girl,"to be that good, it must be taxin' \r no such..."
1630,Olivia Newton-John,I Honestly Love You,maybe i hang around here \r a little more tha...
...,...,...,...
4573,Billy Joel,Worse Comes To Worst,today i'm livin' like a rich man's son \r tom...
1024,Air Supply,I Can Wait Forever,"when you say i miss the things you do, \r i j..."
4274,Jimi Hendrix,Trash Man,i am your trash man- \r throw out all your tr...
218,Black Sabbath,Hot Line,take me to the river baby drink my wine \r wh...


In [45]:
import nltk
from nltk.stem.porter import PorterStemmer
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [46]:
stemmer=PorterStemmer()

In [47]:
def token(txt):
  token = nltk.word_tokenize(txt)
  a=[stemmer.stem(w) for w in token]
  return " ".join(a)


In [48]:
df['text'].apply(lambda x : token(x))

3957    if you ever chang your mind about leav , leav ...
454     toll no bell for me father but let thi cup of ...
431     do what you got ta do with greg allman writer ...
2601    to be that good , it must be taxin ' no such t...
1630    mayb i hang around here a littl more than i sh...
                              ...                        
4573    today i 'm livin ' like a rich man 's son tomo...
1024    when you say i miss the thing you do , i just ...
4274    i am your trash man- throw out all your trash ...
218     take me to the river babi drink my wine when i...
3021    my girlfriend slip and fell now she 's hang fr...
Name: text, Length: 3000, dtype: object

In [49]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [50]:
tfidf= TfidfVectorizer(analyzer='word',stop_words='english')

In [51]:
matrix=tfidf.fit_transform(df['text'])

In [52]:
similer=cosine_similarity(matrix)

In [53]:
similer[0]

array([1.        , 0.01157044, 0.1062713 , ..., 0.02133685, 0.01266354,
       0.00369398])

In [56]:
df[df['song']=='Bring It On Home To Me']

# df[df['song']=='Good Old Days'].index[0]

,artist,song,text
3957,Otis Redding,Bring It On Home To Me,if you ever change your mind \r about leaving...
4881,Van Morrison,Bring It On Home To Me,if you ever (play it slowly) change your mind ...


In [60]:
#recommendet function
def recommender(song_name):
  idx=df[df['song']==song_name].index[0]
  distance=sorted(list(enumerate(similer[idx])),reverse=True,key=lambda x:x[1])
  song=[]
  for s_id in distance[1:20]:
    song.append(df.iloc[s_id[0]].song)
  return song


In [61]:
recommender("Hot Line")

['How Bad Do You Want It?',
 'Bad Girl',
 'Full Moon And Empty Arms',
 'A Song For Richard And His Friends',
 'Weather With You',
 'Bad Seed',
 'Want',
 'Prince Charming',
 'Good Ones And Bad Ones',
 'Trouble',
 'Why Me',
 'Little Earthquakes',
 'All I Hear',
 'Bad Karma',
 'White Lightning',
 'The Mercy Seat',
 'Nasty Naughty Boy',
 'Sound Asleep',
 'Dear Father']

In [65]:
import pickle

In [68]:
pickle.dump(similer,open("similarity","wb"))

In [69]:
pickle.dump(df,open("df","wb"))